In [ ]:
#Use tubodbc for speed
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
#from io import StringIO

from turbodbc import Megabytes, connect, make_options
options = make_options(read_buffer_size=Megabytes(100),
                        parameter_sets_to_buffer=1000,
                        varchar_max_character_limit=10000,
                        use_async_io=True,
                        prefer_unicode=True,
                        autocommit=True,
                        large_decimals_as_64_bit_types=True,
                        limit_varchar_results_to_max=True)

connection = connect(dsn='PROJECT_MT_AUBURN')
cursor = connection.cursor()

## Build Features Data

### Get Data from SQL

In [ ]:
QUERY_1 = '''
SELECT 
    ACCOUNT_NUMBER
	, REPLACE(DX1+','+DX2+','+DX3+','+DX4+','+DX5+','+DX6+','+DX7+','+DX8+','+DX9+','+DX10+','+DX11+','+DX12+','+DX13+','+DX14+','+DX15+','+DX16+','+DX17+','+DX18+','+DX19+','+DX20+','+DX21+','+DX22+','+DX23+','+DX24+','+DX25+','+DX26+','+DX27+','+DX28+','+DX29+','+DX30,',',' ') AS dx
	, REPLACE(PX1+','+PX2+','+PX3+','+PX4+','+PX5+','+PX6+','+PX7+','+PX8+','+PX9+','+PX10+','+PX11+','+PX12+','+PX13+','+PX14+','+PX15,',',' ') as px
	, REPLACE(CPT1+','+CPT2+','+CPT3+','+CPT4+','+CPT5+','+CPT6,',',' ') as cpt
	, REPLACE(MOD1+','+MOD2+','+MOD3+','+MOD4+','+MOD5+','+MOD6+','+MOD7+','+MOD8+','+MOD9+','+MOD10+','+MOD11+','+MOD12+','+MOD13+','+MOD14+','+MOD15,',',' ') as mod
INTO ##CODES
FROM MAH_ENCOUNTER_DETAIL
'''

In [ ]:
cursor.execute(QUERY_1)
connection.commit()

In [ ]:
QUERY_2 = '''
IF OBJECT_ID('TEMPDB.DBO.##TEMP1','U') IS NOT NULL
DROP TABLE ##TEMP1

select
	service_date,
	post_date,
	facility,
	account,
	patient_type,
	insurance_plan,
	payor,
	descrp,
    attending_provider,
	source_ub04,
    hcpcs,
    mod,
	quant,
	price,
    target_ub04,
    labels_dx,
    labels_px,
    labels_cpt,
	labels_mod
into ##TEMP1
from (
		SELECT
			'MAH' as facility
			, CAST(SERVICE_DATE AS DATE) as service_date 
			, CAST(POST_DATE AS DATE) AS post_date
			, 'MAH'+cd.ACCOUNT_NUMBER AS account
			, CASE WHEN ed."Account_Base_Class"='Outpatient' THEN 'O' Else 'I' end as patient_type
			, cd.PRIM_INS_PLAN as insurance_plan
			, cd.PRIM_PAYR as payor
			, LOWER(PROC_DESC) AS descrp
            , ed."Attend_Prov" as attending_provider
            , cds.dx as labels_dx
            , cds.px as labels_px
            , cds.cpt as labels_cpt
            , cds.mod as labels_mod
			, RIGHT(REV_CODE, 3) AS source_ub04
			, ubeditor."Recommendation" as target_ub04
			, QUANTITY AS quant
			, AMOUNT as price
            , LEFT(RIGHT("CPT_CODE", 6), 5) as hcpcs
            , CASE WHEN MODIFIER='' THEN NULL ELSE MODIFIER END AS mod
		FROM PROJECT_MT_AUBURN..MAH_CHARGE_DETAIL cd
		LEFT JOIN CDM..UB_Editor_Recommendations_2017 ubeditor
			on ubeditor."HCPCS" = LEFT(RIGHT("CPT_CODE", 6), 5)
		LEFT JOIN PROJECT_MT_AUBURN..[MAH_ENCOUNTER_DETAIL] ed
			on ed."ACCOUNT_NUMBER" = cd."ACCOUNT_NUMBER"
        LEFT JOIN ##CODES cds
            on cds."ACCOUNT_NUMBER" = cd."ACCOUNT_NUMBER"
		WHERE CPT_CODE IN (select "HCPCS Code" from CDM..ADDENDUMB_2018)
			AND QUANTITY > 0
			AND AMOUNT > 0
			AND "Modifier"  NOT LIKE '%[0-9]%'
		GROUP BY
			CAST(SERVICE_DATE AS DATE) 
			, CAST(POST_DATE AS DATE)
			, cd.ACCOUNT_NUMBER
			, ed."Account_Base_Class"
			, cd.PRIM_INS_PLAN
			, cd.PRIM_PAYR
			, LOWER(PROC_DESC)
            , ed."Attend_Prov"
            , cds.dx
            , cds.px
            , cds.cpt
            , cds.mod
			, RIGHT(REV_CODE, 3)
			, ubeditor."Recommendation"
			, QUANTITY
			, AMOUNT
            , LEFT(RIGHT("CPT_CODE", 6), 5)
            , CASE WHEN MODIFIER='' THEN NULL ELSE MODIFIER END
	) as t
order by
	hcpcs
'''

In [ ]:
cursor.execute(QUERY_2)
connection.commit()

In [ ]:
QUERY_3 ='''
--Calulate average price for CPT/HCPCS Code
SELECT
	a.service_date,
    a.post_date,
	a.facility,
	a.account,
	a.patient_type,
	a.descrp,
	LEN(a.descrp) as descrp_length,
    a.attending_provider,
	a.hcpcs,
	b.count_hcpcs,
	a.mod,
	d.mod_count,
	d.mod_count_quant,
	d.mod_avg_quant,
	d.mod_stdev_quant,
	d.mod_max_quant,
	d.mod_min_quant,
	d.mod_var_quant,
	d.mod_count_price,
	d.mod_avg_price,
	d.mod_stdev_price,
	d.mod_max_price,
	d.mod_min_price,
	d.mod_var_price,
	a.source_ub04,
	c.count_source_ub04,
	a.quant,
	c.source_ub04_avg_quant,
	c.source_ub04_stdev_quant,
	c.source_ub04_max_quant,
	c.source_ub04_min_quant,
	b.hcpcs_count_quant,
	b.hcpcs_avg_quant,
	b.hcpcs_stdev_quant,
	b.hcpcs_max_quant,
	b.hcpcs_min_quant,
	b.hcpcs_var_quant,
	a.price,
	c.source_ub04_count_price,
	c.source_ub04_avg_price,
	c.source_ub04_stdev_price,
	c.source_ub04_max_price,
	c.source_ub04_min_price,
	c.source_ub04_var_price,
	b.hcpcs_count_price,
	b.hcpcs_avg_price,
	b.hcpcs_stdev_price,
	b.hcpcs_max_price,
	b.hcpcs_min_price,
	b.hcpcs_var_price,
    a.target_ub04,
    a.labels_dx,
    a.labels_px,
    a.labels_cpt,
	a.labels_mod
from ##TEMP1 a
left join
	(select
		HCPCS
		, count(hcpcs) as count_hcpcs
		, count(quant) as hcpcs_count_quant
		, avg(quant) as hcpcs_avg_quant
		, stdev(quant) as hcpcs_stdev_quant
		, max(quant) as hcpcs_max_quant
		, min(quant) as hcpcs_min_quant
		, var(quant) as hcpcs_var_quant
		, count(price) as hcpcs_count_price
		, avg(price) as hcpcs_avg_price
		, stdev(price) as hcpcs_stdev_price
		, max(price) as hcpcs_max_price
		, min(price) as hcpcs_min_price
		, var(price) as hcpcs_var_price
	from ##TEMP1
	GROUP BY
		hcpcs
	) as b
ON
	a.hcpcs = b.hcpcs
left join
	(select
		source_ub04
		, count(source_ub04) as count_source_ub04
		, count(quant) as source_ub04_count_quant
		, avg(quant) as source_ub04_avg_quant
		, stdev(quant) as source_ub04_stdev_quant
		, max(quant) as source_ub04_max_quant
		, min(quant) as source_ub04_min_quant
		, count(price) as source_ub04_count_price
		, avg(price) as source_ub04_avg_price
		, stdev(price) as source_ub04_stdev_price
		, max(price) as source_ub04_max_price
		, min(price) as source_ub04_min_price
		, var(price) as source_ub04_var_price
	 from ##TEMP1
	 group by
		source_ub04
	) as c
ON
	a."source_ub04" = c."source_ub04"

left join
	(select
		mod
		, count(mod) as mod_count
		, count(quant) as mod_count_quant
		, avg(quant) as mod_avg_quant
		, stdev(quant) as mod_stdev_quant
		, max(quant) as mod_max_quant
		, min(quant) as mod_min_quant
		, var(quant) as mod_var_quant
		, count(price) as mod_count_price
		, avg(price) as mod_avg_price
		, stdev(price) as mod_stdev_price
		, max(price) as mod_max_price
		, min(price) as mod_min_price
		, var(price) as mod_var_price
	from ##TEMP1
	group by
		mod
	) as d
ON
	a."mod" = d."mod"
		
GROUP BY
	a.service_date,
    a.post_date,
	a.facility,
	a.account,
	a.patient_type,
	a.descrp,
    a.attending_provider,
	a.hcpcs,
	b.count_hcpcs,
	a.mod,
	d.mod_count,
	d.mod_count_quant,
	d.mod_avg_quant,
	d.mod_stdev_quant,
	d.mod_max_quant,
	d.mod_min_quant,
	d.mod_var_quant,
	d.mod_count_price,
	d.mod_avg_price,
	d.mod_stdev_price,
	d.mod_max_price,
	d.mod_min_price,
	d.mod_var_price,
	a.source_ub04,
	c.count_source_ub04,
	a.quant,
	c.source_ub04_avg_quant,
	c.source_ub04_stdev_quant,
	c.source_ub04_max_quant,
	c.source_ub04_min_quant,
	b.hcpcs_count_quant,
	b.hcpcs_avg_quant,
	b.hcpcs_stdev_quant,
	b.hcpcs_max_quant,
	b.hcpcs_min_quant,
	b.hcpcs_var_quant,
	a.price,
	c.source_ub04_count_price,
	c.source_ub04_avg_price,
	c.source_ub04_stdev_price,
	c.source_ub04_max_price,
	c.source_ub04_min_price,
	c.source_ub04_var_price,
	b.hcpcs_count_price,
	b.hcpcs_avg_price,
	b.hcpcs_stdev_price,
	b.hcpcs_max_price,
	b.hcpcs_min_price,
	b.hcpcs_var_price,
    a.target_ub04,
    a.labels_dx,
    a.labels_px,
    a.labels_cpt,
	a.labels_mod
order by
	a.service_date asc
	, account asc
'''

In [ ]:
df1 = pd.read_sql(QUERY_3, connection)
df1.head()

In [ ]:
### Clean up labels columns
df1['labels_dx'] = df1['labels_dx'].str.split()
df1['labels_px'] = df1['labels_px'].str.split()
df1['labels_cpt'] = df1['labels_cpt'].str.split()
df1['labels_mod'] = df1['labels_mod'].str.split()
df1['source_ub04'] = df1['source_ub04'].str.split()
p_cols = ['source_ub04','labels_dx','labels_px','labels_cpt','labels_mod']
df1[p_cols].head()

In [ ]:
### Clean Up Target UB04 columns
df1['target_ub04'] = df1['target_ub04'].str.replace('[^\w\s]','')

numbers_list = []
for string in df1['target_ub04'].astype(str):
    numbers = [int(s) for s in string.split() if s.isdigit()]
    numbers_list.append(numbers)
    
df1['target_ub04'] = numbers_list


In [ ]:
df1['labels_dx'][0]

When converting the labels into a matrix, the size of the matrix can overload the 16 GB ram computer.
To keep the memory size down, will exlude px codes

In [ ]:
df1['labels_combined'] = df1['source_ub04'] +df1['labels_dx']+df1['labels_cpt']+df1['labels_mod']

In [ ]:
df1['labels_combined'][1000]

In [ ]:
df1.head(3)

### Medicare MUE values

In [ ]:
# download data
url = 'https://data.medicaid.gov/api/views/8pny-kgh5/rows.csv?accessType=DOWNLOAD'
mue=pd.read_csv(url)
mue.head()

In [ ]:
# group data
mue_grouped = mue.groupby('HCPCS/CPT Code').agg(['count', 'sum','mean','median','std','min', 'max', 'var'])
mue_grouped.columns = [' '.join(col).strip().lower() for col in mue_grouped.columns.values]
mue_grouped.head()

In [ ]:
# merge data
df2 = df1.set_index('hcpcs')
print(df2.shape[0])
df2 = df2.merge(mue_grouped, how='left', left_index=True, right_index=True)
print(df2.shape[0])

In [ ]:
df2.head(2)

### Medicare Alpha-Numeric 

In [ ]:
'''
# taken out for now, doesn't help much

# download, unzip and load files
import requests, zipfile, io

zip_file_url = 'https://www.cms.gov/Medicare/Coding/HCPCSReleaseCodeSets/Downloads/2018-Alpha-Numeric-HCPCS-File.zip'

r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

alpa_numeric_df = pd.read_excel('HCPC2018_CONTR_ANWEB_disc.xlsx', skiprows=10)


# clean up alpa_numeric
drop_cols = ['SEQNUM',
    'RECID',
    'PRICE2',
    'PRICE3',
    'PRICE4',
    'CIM2',
    'CIM3',
    'MCM2',
    'MCM3',
    'TERM DT',
    'LABCERT2',
    'LABCERT3',
    'LABCERT4',
    'LABCERT5',
    'LABCERT6',
    'LABCERT7',
    'LABCERT8',
    'XREF2',
    'XREF3',
    'XREF4',
    'XREF5',
    'OPPS',
    'OPPS_PI',
    'OPPS_DT',
    'TOS2',
    'TOS3',
    'TOS4',
    'TOS5' 
         ]

alpa_numeric_df = pd.read_excel('HCPC2018_CONTR_ANWEB_disc.xlsx', skiprows=10)
alpa_numeric_df = alpa_numeric_df.drop(columns=drop_cols)
alpa_numeric_df['LONG DESCRIPTION'] = alpa_numeric_df['LONG DESCRIPTION'].str.len()
alpa_numeric_df['SHORT DESCRIPTION'] = alpa_numeric_df['SHORT DESCRIPTION'].str.len()
alpa_numeric_df['ADD DT'] = pd.to_datetime(alpa_numeric_df['ADD DT'],format='%Y%m%d')
alpa_numeric_df['ACT EFF DT'] = pd.to_datetime(alpa_numeric_df['ACT EFF DT'],format='%Y%m%d')
alpa_numeric_df = alpa_numeric_df.set_index('HCPC')
alpa_numeric_df.head(3)

# merge data
print(df2.shape[0])
df2 = df2.merge(alpa_numeric_df, how='left', left_index=True, right_index=True)
print(df2.shape[0])
'''

## Medicare RVU



+ State = Massachusetts
+ Fee Schedule Areas = METROPOLITAN BOSTON
+ Carrier # = 14212
+ Locality Number = 01


In [ ]:
# download, unzip and load files
import requests, zipfile, io

zip_file_url = 'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/PhysicianFeeSched/Downloads/RVU18C1.zip'

r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
medrvu = pd.read_excel('PPRRVU18_JUL.xlsx', skiprows=9)
medrvu = medrvu.set_index('HCPCS')
medrvu.head()

In [ ]:
# merge data
print(df2.shape[0])
df2 = df2.merge(medrvu, how='left', left_index=True, right_index=True)
print(df2.shape[0])
df2.head(3)

In [ ]:
oppscap_df = pd.read_excel('OPPSCAP_JUL.xlsx')
oppscap_df.head(3)

In [ ]:
oppscap_df = oppscap_df[(oppscap_df['LOCALITY'] == 1.0) & (oppscap_df['CARIER'] == 1212.0)]
oppscap_no_mod_df = oppscap_df[oppscap_df['MOD'].isnull()]
oppscap_yes_mod_df = oppscap_df[oppscap_df['MOD'].notnull()]

oppscap_no_mod_df = oppscap_no_mod_df.set_index('HCPCS')
oppscap_yes_mod_df = oppscap_yes_mod_df.set_index('HCPCS')


df2_no_mod = df2[df2['mod'].isnull()]
df2_yes_mod = df2[df2['mod'].notnull()]

In [ ]:
print(df2_no_mod.shape[0])
df2_no_mod = df2_no_mod.merge(oppscap_no_mod_df, how='left', left_index=True, right_index=True).reset_index()
print(df2_no_mod.shape[0])

In [ ]:
print(df2_yes_mod.shape[0])
df2_yes_mod = df2_yes_mod.merge(oppscap_yes_mod_df, how='left', left_index=True, right_index=True).reset_index()
print(df2_yes_mod.shape[0])

In [ ]:
df3 = pd.concat([df2_no_mod, df2_yes_mod])
df3.rename(columns={'index':'hcpcs'}, inplace=True)

In [ ]:
df3.sort_values(by=['service_date','account'], inplace=True)
#df3.reset_index(drop=True, inplace=True)

In [ ]:
df3.head()

In [ ]:
#df3.groupby('BASE').count()

## Prepare Data for ML

In [ ]:
cat_cols = [
'service_date',
'post_date',
'account',
'patient_type',
'attending_provider',
'descrp',
'target_ub04',
'DESCRIPTION',
'CODE',
'PAYMENT',
'IND',
'DAYS',
'MOD_x',
'MOD_y',
'PROCSTAT',
'CARIER',
'LOCALITY',
'OP',
'OP.1',
'OP.2',
'PROC',
'SURG',
'SURG.1',
'SURG.2',
'SURG.3',
'BASE',
'FACTOR',
'PROCEDURES',
'FLAG',
'INDICATOR.2',
]



In [ ]:
numerical_cols = ['descrp_length',
'count_hcpcs',
'mod_count',
'mod_count_quant',
'mod_avg_quant',
'mod_stdev_quant',
'mod_max_quant',
'mod_min_quant',
'mod_var_quant',
'mod_count_price',
'mod_avg_price',
'mod_stdev_price',
'mod_max_price',
'mod_min_price',
'mod_var_price',
'count_source_ub04',
'quant',
'source_ub04_avg_quant',
'source_ub04_stdev_quant',
'source_ub04_max_quant',
'source_ub04_min_quant',
'hcpcs_count_quant',
'hcpcs_avg_quant',
'hcpcs_stdev_quant',
'hcpcs_max_quant',
'hcpcs_min_quant',
'hcpcs_var_quant',
'price',
'source_ub04_count_price',
'source_ub04_avg_price',
'source_ub04_stdev_price',
'source_ub04_max_price',
'source_ub04_min_price',
'source_ub04_var_price',
'hcpcs_count_price',
'hcpcs_avg_price',
'hcpcs_stdev_price',
'hcpcs_max_price',
'hcpcs_min_price',
'hcpcs_var_price',
'mue value count',
'mue value sum',
'mue value mean',
'mue value median',
'mue value std',
'mue value min',
'mue value max',
'mue value var',
'RVU',
'PE RVU',
'PE RVU.1',
'TOTAL',
'TOTAL.1',
'AMOUNT',
'AMOUNT.1',
'AMOUNT.2',
'FACILITY PRICE',
'NON-FACILTY PRICE'
]

In [ ]:
labels_cols = ['labels_combined']

In [ ]:
drop_cols = ['facility','INDICATOR', 'INDICATOR.1']

In [ ]:
df4 = df3.sample(100000, random_state=5)

In [ ]:
df4 = df4.drop(columns=drop_cols)

In [ ]:
cat_df = df4[cat_cols].reset_index(drop=True)
num_df = df4[numerical_cols].reset_index(drop=True)
labels_df = df4[labels_cols].reset_index(drop=True)

In [ ]:
labels_df['labels_combined'][0:3]

### Pre-process Features

In [ ]:
cat_cols = cat_df.columns.tolist()
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
for col in cat_cols:
    cat_df[col] = labelencoder_X_1.fit_transform(cat_df[col].astype(str))

In [ ]:
cat_df.shape[0]

In [ ]:
cat_df.head()

In [ ]:
index = cat_df.index.values.tolist() 

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(cat_df).toarray()
#index = [str(i) for i in range(1, len(X)+1)]
cat_hot = pd.DataFrame(X, index=index)
cat_hot.head()

In [ ]:
cat_hot.shape[0]

In [ ]:
num_df = num_df.fillna(0.0)

In [ ]:
num_df.shape[0]

In [ ]:
num_df.head()

In [ ]:
cat_hot.head()

In [ ]:
num_df.tail()

In [ ]:
cat_hot.tail()

In [ ]:
del cat_df
del df4
del df3
del df2
del df1

In [ ]:
cat_hot.to_csv("cat_hot.zip", index_label=False, chunksize=10000, compression='zip')

In [ ]:
del cat_hot

In [ ]:
num_df.to_csv("num_df.zip", index_label=False, chunksize=10000, compression='zip')

In [ ]:
del num_df

In [ ]:
labels_df.to_csv("labels_df.zip", index_label=False, chunksize=10000, compression='zip')

In [ ]:
del labels_df

### \*Restart the Kernel for memory reasons and begin running from here

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv("cat_hot.zip", nrows=3, compression='zip')
df2 = pd.read_csv("num_df.zip", nrows=3, compression='zip')

In [ ]:
# creating a empty bucket to save result
df_result = pd.DataFrame(columns=(df1.columns.append(df2.columns)).unique())
df_result.to_csv('features.csv',index_label=False)

In [ ]:
# deleting df2 to save memory
del(df2)
del(df1)

In [ ]:
def preprocess(x):
    df2=pd.merge(df1,x,how='inner', left_index=True, right_index=True)
    df2.to_csv("features.csv",mode="a",header=False,index=False)   

In [ ]:
reader = pd.read_csv("cat_hot.zip", chunksize=1000, compression='zip') # chunksize depends with you colsize

In [ ]:
df1 = pd.read_csv('num_df.zip', compression='zip')

In [ ]:
[preprocess(r) for r in reader]

In [ ]:
#features = cat_hot.merge(num_df, how='inner', left_index=True, right_index=True)

### Pre-Process Labels (Restart Kernel Again)

In [ ]:
import pandas as pd

In [ ]:
labels_df = pd.read_csv("labels_df.zip", compression='zip')

from ast import literal_eval
new_list = []
s = labels_df['labels_combined']
for i in s:
    new_string = i
    #print(literal_eval(new_string))
    new_list.append(literal_eval(new_string))

labels_df['labels_combined'] = new_list

In [ ]:
del new_list
del s

In [ ]:
labels_df['labels_combined'][0]

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = labels_df['labels_combined'].tolist()
y[0]

In [ ]:
del labels_df

In [ ]:
y = mlb.fit_transform(y)

In [ ]:
print("rows ",y.shape[0])
print("columns ",y.shape[1])
print("cells ", y.shape[0]*y.shape[1])

## Split Train and Test

In [ ]:
features = pd.read_csv('features.csv')

In [ ]:
features.shape[0]

In [ ]:
y.shape[0]

In [ ]:
labels_list = list(mlb.classes_)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.33, random_state=42)

In [ ]:
import hickle as hkl
hkl.dump(X_train, 'X_train.hkl' )
hkl.dump(X_test, 'X_test.hkl' )
hkl.dump(y_test, 'y_test.hkl' )
hkl.dump(y_train, 'y_train.hkl' )
hkl.dump(labels_list, 'labels_list.hkl' )